## Task 1

In [ ]:
class SudokuSolver:
    def __init__(self, grid):
        self.grid = grid
        self.variables = [(i, j) for i in range(9) for j in range(9)]
        self.domains = {
            var: set(range(1, 10)) if grid[var[0]][var[1]] == 0 else {grid[var[0]][var[1]]}
            for var in self.variables
        }
        self.constraints = self.generate_constraints()

    def generate_constraints(self):
        """
        Generate constraints mapping for each cell.
        Each key in the dictionary is a cell (i, j), and its value is a set of cells
        that are in the same row, column, or subgrid (3x3 block) as (i, j).
        """
        constraints = {}
        for i in range(9):
            for j in range(9):
                neighbors = set()
                # Add all cells in the same row
                for col in range(9):
                    if col != j:
                        neighbors.add((i, col))
                # Add all cells in the same column
                for row in range(9):
                    if row != i:
                        neighbors.add((row, j))
                # Add all cells in the same 3x3 subgrid
                block_row_start = (i // 3) * 3
                block_col_start = (j // 3) * 3
                for r in range(block_row_start, block_row_start + 3):
                    for c in range(block_col_start, block_col_start + 3):
                        if (r, c) != (i, j):
                            neighbors.add((r, c))
                constraints[(i, j)] = neighbors
        return constraints


    def is_valid(self, var, value):
        """
        Check if assigning 'value' to 'var' satisfies all constraints.
        """
        # TODO: Check against other variables in the same constraint group
        neighbors = self.constraints[var]
        for neighbor in neighbors:
            if self.grid[neighbor[0]][neighbor[1]] == value:
                return False
        return True

    def forward_checking(self, var, value):
        """
        Remove 'value' from the domain of neighbors of 'var'.
        Return list of affected variables to restore later.
        """
        affected = []
        for neighbor in self.constraints[var]:
            if self.grid[neighbor[0]][neighbor[1]] == 0:
                if value in self.domains[neighbor]:
                    self.domains[neighbor].remove(value)
                    affected.append(neighbor)
        return affected


    def restore_domains(self, affected, value):
        """
        Restore removed values after backtracking.
        """
        # TODO: Restore domain values to affected variables
        for var in affected:
            self.domains[var].add(value)

    def select_unassigned_variable(self):
        unassigned = [var for var in self.variables if self.grid[var[0]][var[1]] == 0]
        
        if not unassigned:
            return None
        
        return min(unassigned, key=lambda var: len(self.domains[var]))


    def backtrack(self):
        if all(self.grid[i][j] != 0 for i in range(9) for j in range(9)):
            return True

        var = self.select_unassigned_variable()

        for value in list(self.domains[var]):
            if self.is_valid(var, value):
                self.grid[var[0]][var[1]] = value
                original_domain = self.domains[var]
                self.domains[var] = {value}

                affected = self.forward_checking(var, value)

                if self.backtrack():
                    return True

                self.grid[var[0]][var[1]] = 0
                self.domains[var] = original_domain
                self.restore_domains(affected, value)

        return False

    def solve(self):
        if self.backtrack():
            return self.grid
        else:
            return None

In [8]:

# 9x9 Sudoku Grid (0 represents empty cells)
sudoku_grid = [
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9],
]

# Solve the Sudoku
solver = SudokuSolver(sudoku_grid)
solution = solver.solve()

# Print the solution
if solution:
    print("Solved Sudoku:")
    for row in solution:
        print(row)
else:
    print("No solution exists.")


Solved Sudoku:
[5, 3, 4, 6, 7, 8, 9, 1, 2]
[6, 7, 2, 1, 9, 5, 3, 4, 8]
[1, 9, 8, 3, 4, 2, 5, 6, 7]
[8, 5, 9, 7, 6, 1, 4, 2, 3]
[4, 2, 6, 8, 5, 3, 7, 9, 1]
[7, 1, 3, 9, 2, 4, 8, 5, 6]
[9, 6, 1, 5, 3, 7, 2, 8, 4]
[2, 8, 7, 4, 1, 9, 6, 3, 5]
[3, 4, 5, 2, 8, 6, 1, 7, 9]
